In [ ]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub langchain langgraph tavily-python llama-index llama-parse llama-index-llms-groq llama-index-embeddings-fastembed fastembed  llama-index-vector-stores-qdrant

In [7]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [13]:
import PyPDF2

def extract_pages(input_pdf, output_pdf, start_page, end_page):

    with open(input_pdf, 'rb') as infile:
        pdf_reader = PyPDF2.PdfReader(infile)
        pdf_writer = PyPDF2.PdfWriter()

        # Ensure the start and end pages are within the range of available pages
        if start_page < 0 or end_page >= len(pdf_reader.pages) or start_page > end_page:
            raise ValueError("Page range is out of bounds.")

        # Add specified pages to the writer
        for page_num in range(start_page, end_page + 1):
            pdf_writer.add_page(pdf_reader.pages[page_num])

        # Write the output PDF file
        with open(output_pdf, 'wb') as outfile:
            pdf_writer.write(outfile)

# Define the input PDF, output PDF, and page range to extract (0-based index)
input_pdf_path = '/content/drive/MyDrive/Davidson Medicine 24th.pdf'
output_pdf_path = '/content/cardio_vascular.pdf'
start_page = 428
end_page = 523

extract_pages(input_pdf_path, output_pdf_path, start_page, end_page)

In [14]:
import os
import nest_asyncio
nest_asyncio.apply()
os.environ['LLAMA_PARSE_API_KEY']="xxxxxxxxxxx"

from llama_parse import LlamaParse
llama_parse_documents = LlamaParse(api_key="xxxxxxxxxxx", result_type="markdown").load_data("/content/cardio_vascular.pdf")

Started parsing the file under job_id d7e648e6-cbd5-4610-a92e-a515b2647d31
.......

In [15]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

In [16]:
# model
from llama_index.llms.groq import Groq

os.environ["GROQ_API_KEY"]='xxxxxxxxxxx'
llm1 = Groq(model="mixtral-8x7b-32768", api_key="xxxxxxxxxxx")

In [17]:
from llama_index.core import Settings
Settings.llm=llm1
Settings.embed_model=embed_model

In [18]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
import qdrant_client

client = qdrant_client.QdrantClient(
    "qdrant_endpoint", # you can obtain this from qdrant cloud 
    api_key="xxxxxxxxxxx",  #qdrant api key
)

vector_store = QdrantVectorStore(client=client, collection_name="health_documents")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index =VectorStoreIndex.from_documents(documents=llama_parse_documents,storage_context=storage_context,show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/100 [00:00<?, ?it/s]

In [19]:
from IPython.display import display, Markdown

In [44]:
query_engine = index.as_query_engine()
response = query_engine.query("some of the causes and prevention of diabetes?")
display(Markdown(response.response))

Diabetes is a complex disease that can be caused by a combination of genetic and lifestyle factors. While the information provided does not directly mention diabetes, it does discuss some relevant topics.

One of the risk factors for diabetes is obesity, which is also associated with a higher risk of cardiovascular disease. Primary prevention strategies, such as modifying diet and exercise, can help reduce the risk of both cardiovascular disease and diabetes.

In the context of hypertension, which is a common complication of diabetes, accurate blood pressure measurement is crucial. This involves using a validated machine, supporting the arm at the level of the heart, and using a cuff of appropriate size.

While not directly mentioned in the provided context, it's important to note that maintaining a healthy weight, engaging in regular physical activity, and following a balanced diet can all contribute to the prevention and management of diabetes. Regular check-ups and early detection can also help control the disease and prevent complications.

In [58]:
retriever = index.as_retriever(search_kwargs={"k":5})

In [24]:
os.environ['LANGCHAIN_TRACING-V2']='true'
os.environ['LANGCHAIN_ENDPOINT']='https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']='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'

In [25]:
%pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.2 MB/s eta 0:00:00


In [27]:
import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

In [28]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    model="mixtral-8x7b-32768",
    api_key="xxxxxxxxxxx"
)

In [59]:
# generation
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.Only
    Use the following pieces of retrieved context to answer the question.If context doesnot provide information to answer that question, just say user to do web search.
    If you find it,try to explain the answer keeping the answer concise. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
start = time.time()
rag_chain = prompt | llm | StrOutputParser()
question = "what medicines are recommended for diabetes?"
docs = retriever.retrieve(question)
docs = [d.text for d in docs]
generation = rag_chain.invoke({"context":docs,"question": question})
print(generation)
end = time.time()
print(f"The time required to generate response by Router Chain in seconds:{end - start}")

The context provided does not include specific information about medications recommended for diabetes. I recommend doing a web search for "medications recommended for diabetes" to get the most accurate and up-to-date information.
The time required to generate response by Router Chain in seconds:1.2609198093414307


In [45]:
# answer grader
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an
    answer is useful to resolve a question. Give a  score 'yes' or 'no' to indicate whether the answer is
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation}
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)
start = time.time()
answer_grader = prompt | llm | JsonOutputParser()
answer_grader_response = answer_grader.invoke({"question": question,"generation": generation})
end = time.time()
print(f"The time required to generate response by the answer grader in seconds:{end - start}")
print(answer_grader_response)

The time required to generate response by the answer grader in seconds:0.3321113586425781
{'score': 'yes'}


In [46]:
# web search
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['TAVILY_API_KEY'] = "xxxxxxxxxxx"
web_search_tool = TavilySearchResults(k=3)

In [47]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    question : str
    generation : str
    web_search : str
    documents : List[str]

In [48]:
from langchain.schema import Document

In [49]:
def generate(state):

  question = state["question"]
  documents = state["documents"]

  query_engine = index.as_query_engine()
  # generation
  generation = query_engine.query(question)
  generation = generation.response
  print("GENERATING FROM CONTEXT")
  # using answer grader to grade our answer
  score = answer_grader.invoke({"question": question,"generation": generation})
  print(score['score'])
  if score['score']=='yes':
    print(" CONTEXT RESPONSE IS OK")
    return {"documents": documents, "question": question,"generation":generation}

  else:
    #state["web_search"]=='yes'
    print("DOING WEB SEARCH")
    while score['score']=='no':

      question=state["question"]
      documents=state["documents"]
      # web searching
      docs = web_search_tool.invoke({"query": question})
      print(docs)
      web_results = "\n".join([d["content"] for d in docs])
      web_results = Document(page_content=web_results)
      if documents is not None:
          documents.append(web_results)
      else:
          documents = [web_results]


      # generation
      generation = rag_chain.invoke({"context": documents, "question": question})
      print("GENERATING FROM WEB_SEARCH")
      # grading answer
      score = answer_grader.invoke({"question": question,"generation": generation})
      if score['score']=='yes':
        print("WEBSEARCH RESULT IS OK")
        return {"documents": documents, "question": question,"generation":generation}
        break


In [50]:
from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

#workflow.add_node("websearch",web_search)
workflow.add_node("generate",generate)

In [51]:
workflow.set_entry_point("generate")
#workflow.add_edge("generate","websearch")
workflow.set_finish_point("generate")

In [52]:
app = workflow.compile()

In [69]:
from pprint import pprint
inputs = {"question":"how blood pressure level varies with ages."}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
# pprint(Markdown(value["generation"]))
display(Markdown(value["generation"]))

GENERATING FROM CONTEXT
no
DOING WEB SEARCH
[{'url': 'https://www.hriuk.org/health/learn/risk-factors/what-is-normal-blood-pressure-by-age', 'content': 'Greater than 90. Around 120/80 mm Hg is often considered a typical blood pressure reading. Below is the average blood pressure for children and teens: Age. Systolic (top number) mm Hg. Diastolic (bottom number) mm Hg. Newborns up to 1 month. 60-90. 20-60.'}, {'url': 'https://www.forbes.com/health/blood-pressure/normal-blood-pressure-chart-by-age/', 'content': "While numbers lower than 120/80 are generally considered normal, Dr. Desai adds, “The target blood pressure for treatment varies depending on age (e.g. if someone is considered elderly) and associated co-morbidities (e.g. diabetes).”\nNormal Blood Pressure by Age, Gender, Race and Ethnicity\nPreviously, guidance for normal blood pressure for adults varied by gender and specific age, but new data states normal blood pressure for adults as a collective is less than 120/80 mmHg[2]Hi

Blood pressure levels can vary with ages. In general, blood pressure tends to increase with age due to stiffening of the blood vessels. For children and teens, the average blood pressure varies by age. For adults, a normal blood pressure is generally considered to be less than 120/80 mmHg. However, the target blood pressure for treatment can vary depending on age and associated health conditions. For instance, older adults or those with conditions like diabetes may have different target blood pressure levels.